# Introduction

This notebook is for visualisation of results and plotting.

In [ ]:
import matplotlib.pyplot as plt
from locpix.preprocessing import datastruc
from locpix.visualise import vis_img
from locpix.img_processing import render
import os
import numpy as np
import json
import seaborn as sns
import pandas as pd

## Raw images

In [ ]:
file = "egfr_568_ereg_647_fov1"
channel = "egfr"
threshold = 0
interpolation = 'log2'
alphas = [1,0,0,0]
alpha_seg = 0
cmap_seg = ["k", "y"]
figsize = [10,10]
four_colour = False
background_one_colour = False
legend=False

In [ ]:
# folders
input_folder = "annotate/annotated"

# load item
item = datastruc.item(None, None, None, None, None)
item.load_from_parquet(os.path.join(input_folder, file+'.parquet'))

# render histogram & convert to img
histo, channel_map, label_map = item.render_histo([channel])
print(label_map)
img = np.transpose(histo, (0, 2, 1))

# only visualise one channel
vis_img.visualise_seg(
    img,
    np.ones((500,500)),
    item.bin_sizes, 
    axes=[0],
    label_map=label_map,
    threshold=threshold,
    how=interpolation,
    alphas=alphas,
    blend_overlays=False,
    alpha_seg=alpha_seg,
    cmap_img=None,
    cmap_seg=cmap_seg,
    figsize=figsize,
    origin="upper",
    save=False,
    save_loc=None,
    four_colour=four_colour,
    background_one_colour=background_one_colour,
    legend=legend,
    show=True,
)


## Annotation

In [ ]:
file = "egfr_568_ereg_647_fov1"
channel = "egfr"
threshold = 0
interpolation = 'log2'
alphas = [1,.5,.2,.1]
alpha_seg = 0.8
cmap_seg = ["k", "y"]
figsize = [10,10]
four_colour = False
background_one_colour = False
legend=False
column_vis="gt_label"

In [ ]:
# folders
input_folder = "annotate/annotated"

# load item
item = datastruc.item(None, None, None, None, None)
item.load_from_parquet(os.path.join(input_folder, file+'.parquet'))

# render histogram & convert to img
histo, channel_map, label_map = item.render_histo([channel])
print(label_map)
img = np.transpose(histo, (0, 2, 1))

# render segmentation
item.histo_mask = item.render_seg()

# only visualise one channel
vis_img.visualise_seg(
    img,
    item.histo_mask.T,
    item.bin_sizes, 
    axes=[0],
    label_map=label_map,
    threshold=threshold,
    how=interpolation,
    alphas=alphas,
    blend_overlays=False,
    alpha_seg=alpha_seg,
    cmap_img=None,
    cmap_seg=cmap_seg,
    figsize=figsize,
    origin="upper",
    save=False,
    save_loc=None,
    four_colour=four_colour,
    background_one_colour=background_one_colour,
    legend=legend,
    show=True,
)

## Membrane prob map

## Cell segmentation

In [ ]:
file = "egfr_647_ereg_568_fov4"
channel = "egfr"
threshold = 0
interpolation = 'log2'
alphas = [1,.5,.2,.1]
alpha_seg = 0.5
# cmap_seg = ["k", "y"]
figsize = [10,10]
four_colour = True
background_one_colour = False
legend=False
column_vis="pred_label"
method = "classic"

In [ ]:
# folders
input_img_folder = f"{method}/cell/seg_dataframes"
input_seg_folder = f"{method}/cell/seg_img"

# load item
item = datastruc.item(None, None, None, None, None)
item.load_from_parquet(os.path.join(input_img_folder, file+'.parquet'))
item._coord_2_pixel()

# render histogram & convert to img
histo, channel_map, label_map = item.render_histo([channel])
print(label_map)
img = np.transpose(histo, (0, 2, 1))

# load segmentation
segmentation = np.load(os.path.join(input_seg_folder, file+'.npy'))

# only visualise one channel
vis_img.visualise_seg(
    img,
    segmentation,    
    item.bin_sizes, 
    axes=[0],
    label_map=label_map,
    threshold=threshold, 
    how=interpolation, 
    # alphas=alphas, 
    blend_overlays=True,
    alpha_seg=alpha_seg,
    # cmap_img=None, 
    # cmap_seg=cmap_seg, 
    figsize=figsize, 
    origin="upper",
    save=False,
    save_loc=None,
    four_colour=four_colour, 
    #background_one_colour=background_one_colour, 
    legend=legend,
    show=True,
)

## Membrane segmentation

In [ ]:
method = "ilastik"
test_or_val = "test"
fold = 0
file = "ereg_568_egfr_647_fov1"

channel = "egfr"
threshold = 0
interpolation = 'log2'
alphas = [1,.5,.2,.1]
alpha_seg = 0.8
cmap_seg = ["k", "y"]
figsize = [10,10]
four_colour = False
background_one_colour = False
legend=False
column_vis="pred_label"

In [ ]:
# folders
input_folder = f'membrane_performance_arxiv/{method}/membrane/seg_dataframes/{test_or_val}/{fold}'

# load item
item = datastruc.item(None, None, None, None, None)
item.load_from_parquet(os.path.join(input_folder, file+'.parquet'))
item._coord_2_pixel()

# render histogram & convert to img
histo, channel_map, label_map = item.render_histo([channel])
print(label_map)
img = np.transpose(histo, (0, 2, 1))

# render segmentation
item.histo_mask = render.visualise_seg_item(item, column_vis)

# only visualise one channel
vis_img.visualise_seg(
    img,
    item.histo_mask.T,
    item.bin_sizes, 
    axes=[0],
    label_map=label_map,
    threshold=threshold,
    how=interpolation,
    alphas=alphas,
    blend_overlays=False,
    alpha_seg=alpha_seg,
    cmap_img=None,
    cmap_seg=cmap_seg,
    figsize=figsize,
    origin="upper",
    save=False,
    save_loc=None,
    four_colour=four_colour,
    background_one_colour=background_one_colour,
    legend=legend,
    show=True,
)

## PR curves

For each fold have one PR curve for train, test, val each with all methods overlaid

In [ ]:
folder = "membrane_performance"
fold = 0
split = "test"

In [ ]:
data = {}

for method in ["classic", "cellpose_no_train", "cellpose_train", "ilastik"]:

    print('method ', method)


    files_loc = f"{folder}/{method}/membrane/metrics/{fold}"
    files = os.listdir(files_loc)
    for file in files:
        if file.startswith('test_'):
            metrics_loc = os.path.join(files_loc, file)

    with open(metrics_loc, 'r') as f:
        for no, line in enumerate(f):
            if line.startswith("prcurve_pr"):
                prec = json.loads(line[13:])
            if line.startswith("prcurve_rec"):
                rec = json.loads(line[14:])
            if line.startswith("prcurve_baseline"):
                baseline = float(line[19:])

    # remove duplicate prec/recall value
    prec = np.array(prec).round(2)
    rec = np.array(rec).round(2)
    unique_df = np.unique(np.column_stack([prec,rec]), axis=0)
    prec = unique_df[:,0]
    rec = unique_df[:,1]

    data[method] = {'Precision':prec, 'Recall':rec, 'baseline':baseline}

# plot 
sns.set_style("darkgrid")

classic_baseline = data['classic']['baseline']
data['classic'].pop('baseline',None)
classic_df = pd.DataFrame(data=data['classic'])
classic_df['Method'] = 'Classic'

cellpose_no_train_baseline = data['cellpose_no_train']['baseline']
data['cellpose_no_train'].pop('baseline',None)
cellpose_no_train_df = pd.DataFrame(data=data['cellpose_no_train'])
cellpose_no_train_df['Method'] = 'Cellpose (no train)'

cellpose_train_baseline = data['cellpose_train']['baseline']
data['cellpose_train'].pop('baseline',None)
cellpose_train_df = pd.DataFrame(data=data['cellpose_train'])
cellpose_train_df['Method'] = 'Cellpose (trained)'

ilastik_baseline = data['ilastik']['baseline']
data['ilastik'].pop('baseline',None)
ilastik_df = pd.DataFrame(data=data['ilastik'])
ilastik_df['Method'] = 'Ilastik'

assert(classic_baseline==cellpose_no_train_baseline)
assert(cellpose_train_baseline==ilastik_baseline)
assert(classic_baseline==cellpose_train_baseline)
baseline = classic_baseline

df = pd.concat([classic_df, cellpose_no_train_df, cellpose_train_df, ilastik_df])

sns.lineplot(x='Recall', y='Precision', data=df, hue='Method')
plt.plot([0, 1], [baseline, baseline], color="navy", lw=1, linestyle="--")
#plt.plot([0, 1], [1, 1], color="#D41159", lw=1, linestyle="--")
#plt.plot([1, 1], [0, 1], color="#D41159", lw=1, linestyle="--")
#sns.lineplot(x='rec', y='prec', data=cellpose_no_train_df)
#sns.lineplot(x='rec', y='prec', data=cellpose_train_df)
#sns.lineplot(x='rec', y='prec', data=ilastik_df)
    


## Aggregated metrics

In [ ]:
folder = "membrane_performance"
method = 'ilastik'
split = "test"

In [ ]:
iou_0 = []
iou_1 = []
recall_0 = []
recall_1 = []
prec_0 = []
prec_1 = []
macc_list = []
miou_list = []
pr_auc_list = []
aucnpr_list = []
f_0 = []
f_1 = []


for fold in range(5):

    if fold!=0:
        continue
    
    files_loc = f"{folder}/{method}/membrane/metrics/{fold}"
    files = os.listdir(files_loc)
    for file in files:
        if file.startswith('test_'):
            metrics_loc = os.path.join(files_loc, file)

    with open(metrics_loc, 'r') as f:
        for no, line in enumerate(f):
            if no == 2:
                iou_list = json.loads(line[11:])
                iou_0.append(iou_list[0])
                iou_1.append(iou_list[1])

            elif no == 4:
                recall_list = json.loads(line[14:])
                recall_0.append(recall_list[0])
                recall_1.append(recall_list[1])

            if no == 5:
                prec_list = json.loads(line[17:])
                prec_0.append(prec_list[0])
                prec_1.append(prec_list[1])

            elif no == 6:
                macc = float(line[7:])
                macc_list.append(macc)
            
            elif no == 7:
                miou = float(line[7:])
                miou_list.append(miou)
            
            elif no == 9:
                f_list = json.loads(line[11:])
                f_0.append(f_list[0])
                f_1.append(f_list[1])

            elif no == 11:
                pr_auc = float(line[10:])
                pr_auc_list.append(pr_auc)

            elif no == 15:
                aucnpr = float(line[9:])
                aucnpr_list.append(aucnpr)

# return method results
print('Method', method)
print('Iou_0 ', np.mean(iou_0), ' +/- ', np.std(iou_0))
print('Iou_1 ', np.mean(iou_1), ' +/- ', np.std(iou_1))
print('recall_0 ', np.mean(recall_0), ' +/- ', np.std(recall_0))
print('recall_1 ', np.mean(recall_1), ' +/- ', np.std(recall_1))
print('prec_0 ', np.mean(prec_0), ' +/- ', np.std(prec_0))
print('prec_1 ', np.mean(prec_1), ' +/- ', np.std(prec_1))
print('f_0 ', np.mean(f_0), ' +/- ', np.std(f_0))
print('f_1 ', np.mean(f_1), ' +/- ', np.std(f_1))
print('macc ', np.mean(macc_list), ' +/- ', np.std(macc_list))
print('miou ', np.mean(miou_list), ' +/- ', np.std(miou_list))
print('pr_auc', np.mean(pr_auc_list), ' +/- ', np.std(pr_auc_list))
print('aucnpr', np.mean(aucnpr_list), ' +/- ', np.std(aucnpr_list))